# Part 04 - Errors (Fantastic exceptions and where to find them) and File I/O

*Many of the examples in this notebook have deliberate errors. When Jupyter encounters an error in a cell, it stops running all further cells, so you will have to rerun each cell yourself. A shortcut is to select the cell and press shift-enter.* 

# Exceptions/Errors

As you're new programmers, you will spend most of your time handling errors. Sometimes, you even want to make errors of your own. Fortunately, you can teach python to handle errors or make them for you. Let's deliberately try something stupid and see what happens,

In [1]:
import math
#logarithms of negative numbers are not valid
math.log(-1) 

ValueError: math domain error

Here we see something called `ValueError`, which has a `math domain error` message associated with it. Where did this come from?

When bad values are passed to a function, the designer of the function can't get it to return a value as that might be misinterpreted as a normal answer by accident. Instead, we design the functions to raise an exception. Exceptions are not return values but instead bubble up the stack until they are caught or they come out of the main scope where they cause python to exit.

The stack is all the functions that python has had to enter to get to the current line of execution, its how python tracks where return values should actually be sent. Going up the stack, python looks for anything that might catch the exception. Lets see how an exception can be caught, 

In [2]:
import math
try:
    math.log(-1)
except Exception as e:
    print("Exception has been caught! Details below")
    print(e)
    print(type(e))
    print(repr(e))

Exception has been caught! Details below
math domain error
<class 'ValueError'>
ValueError('math domain error')


If any commands within a `try` block raise an exception, and the exception rises up to the `try` block then it will be caught and the code in the `except:` block is run. Above, we just print the exception, the type of the exception, and we use the `repr` command which returns the a python command which will generate the object passed as an argument.

The `try...catch` block above catches all exceptions (as all exceptions must be built/inherited from the `Exception` class). Generally its "bad practice" to catch all exceptions this way, as maybe we will catch the wrong type of error by mistake which we don't know how to handle. We can actually catch specfic exceptions and deal with them in a special way,

In [3]:
try:
    math.log(0)
except ValueError as e:
    print("Bad value used, did you use log(x) where x <= 0?")

Bad value used, did you use log(x) where x <= 0?


How do we raise exceptions ourselves? We just write raise, then use an exception type:

In [4]:
def f():
    raise RuntimeError("My own exception")
    
def g():
    f()

g()

RuntimeError: My own exception

A `RuntimeError` is a generic exception that takes an error message. Most of the time you should use a more specific one like `ValueError` (or make your own, but we need classes for that). Notice that the exception shows you the stack where the interpreter/python was when the exception was raised, from top to bottom function scope.

Lets look how you might use it to improve our prime number function,

In [5]:
import math
def isPrime(n):
    # Perform sanity checks on the input, if its a float, try to convert it into an int
    if isinstance(n, float):
        if n.is_integer():
            n = int(n)
        else:
            raise ValueError("isPrime cannot accept fractional floating values")
    
    if not isinstance(n, int):
        raise ValueError("isPrime can only accept integer arguments")
        
    for div in range(2,int(math.sqrt(n))):
        if n % div == 0:
            return False
    return True

print(2**31-1, isPrime(2**31-1))
print(3.0, isPrime(3.0))

#Bad usages of isPrime() which raise exceptions! Comment out the first line to see the second error
isPrime(2.2)
#isPrime("Not an integer")

2147483647 True
3.0 True


ValueError: isPrime cannot accept fractional floating values

We now get nice error messages when we abuse our `isPrime` function.

## File I/O

Now we know about exceptions, we can actually take a look at file Input/Output.

Opening a file is very easy, and once its open we can even write to it:

In [7]:
# Open a file called myfile.txt for (w)riting
with open('myfile.txt', 'w') as output:
    print("The first line in the file", file=output)
    print("The second line in the file", file=output)

The `with` block is a way of making sure that the next bit of code ONLY happens if the file can be opened. Also, it guarantees that if anything goes wrong inside the `with` block (and an exception is raised), the file will be closed before the exception escapes the `with` block (although the file will be closed anyway by garbage collection if python eventually exits). 

We could just write `output = open('myfile.txt')` and it would work, but the use of with is better at handling errors. Good programming is all about robust code that fails gracefully.

If you want to see the file that you've written out by running the above code, then go to the home page of the notes/jupyter notebook and take a look.

Now we have a file written out, how do we read it?

In [8]:
# Now its open for (r)eading
with open('myfile.txt', 'r') as lines:
    try:
        while True:
            print(next(lines))
    except StopIteration:
        print('EOF!')

The first line in the file

The second line in the file

EOF!


The `while` statement repeats the block as long as the expression to its right is `True`. Here, the expression is `True` so the block is repeated indefinitely. The only way that block can end is if an exception is raised. Fortunately, when the file runs out of lines, the `next()` function will throw a `StopIteration` exception, which we catch to say End Of File (EOF)! This is a case where we MUST use exceptions as part of our normal program execution (its not an error to get a `StopIteration`).

If you notice the results from the above code, we have extra lines appearing, where do these come from? Lets take another look but this time using `repr`

In [9]:
with open('myfile.txt', 'r') as lines:
    try:
        while True:
            print(repr(next(lines)))
    except StopIteration:
        print('EOF!')

'The first line in the file\n'
'The second line in the file\n'
EOF!


Here we can see that each line is being read from the file including its terminating newline character. When we print this, both the terminating newline `\n` and one automatically added by `print` is being output resulting in a double line break. We can fix this by telling print not to end the line that way:

In [10]:
with open('myfile.txt', 'r') as lines:
    try:
        while True:
            print(next(lines), end='')
    except StopIteration:
        print('EOF!')

The first line in the file
The second line in the file
EOF!


There is another way to access the lines of a file, a bit neater, and useful if every line in your file can be processed identically:

In [14]:
with open('myfile.txt', 'r') as lines:
    for line in lines:
        print(line, end='')

The first line in the file
The second line in the file


## Better file IO with pickle

Often you don't want to save text, you want to save the array or list you've been working on. Python has a cool library for that called pickle. 

In [11]:
import pickle
favorite_color = { "lion": "golden", "kitty": "ginger" }
pickle.dump( favorite_color, open("save.p", "wb"))

We've been fast and loose here, not using a `with`, but sometimes we like to program dangerously as time is short. You might notice the `'b'` added to the `open` command, this opens the file in binary mode, which is very space efficient but not readable by humans. You probably should only do this if you're pickling lots of data.

Now, if we want to reload this, we just do the following:

In [12]:
favorite_color_loaded = pickle.load( open( "save.p", "rb" ) )
print(favorite_color_loaded)

{'lion': 'golden', 'kitty': 'ginger'}
